<a href="https://colab.research.google.com/github/rkuo2000/GenAI/blob/main/Colab/pyngrok_LLM_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM HTTP Server
## using pyNgrok for TCP tunneling

In [ ]:
!pip install flask
!pip install pyngrok

## LLM model

In [ ]:
import transformers
from transformers import AutoModelForCausalLM , AutoTokenizer
import torch
torch.set_default_device("cuda")

#model_name = "Q-bert/Mamba-130M"
#model_name = "Q-bert/Mamba-370M"
#model_name = "Q-bert/Mamba-790M"
#model_name = "Q-bert/Mamba-1B"
#model_name = "Q-bert/Mamba-3B"
#model_name = "Q-bert/Mamba-3B-slimpj"
#model_name = "ckip-joint/bloom-3b-zh"
#model_name = "Qwen/Qwen1.5-7B-Chat"
#model_name = "lmsys/vicuna-7b-v1.5-16k"
#model_name = "yentinglin/Taiwan-LLM-7B-v2.0.1-chat"
#model_name = "mistralai/Mistral-7B-Instruct-v0.2"
model_name = "MediaTek-Research/Breeze-7B-Instruct-v0.1"

LLM = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype="auto", device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)

## HTTP Server with Ngrok

In [ ]:
import getpass
import os
import threading

from flask import Flask, request, jsonify
import json

from pyngrok import ngrok, conf

## set ngrok authtoken
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

app = Flask(__name__)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(8000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 8000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Define Flask routes
@app.route("/")
def index():
    return "Hello from Colab!"

@app.route("/text", methods=['POST'])
def text():
    if request.method == 'POST':
        prompt = request.json['text']
        print(prompt)

        input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        output = LLM.generate(input_ids, max_length=64, num_beams=5, no_repeat_ngram_size=2)
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        print("LMM = "+mode_name+"\n")
        print(generated_text)
        return jsonify(generated_text)

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()